### Restore graph from ckpt model

In [2]:
import tensorflow as tf
import numpy as np

### Restore Graph using topo algorithm

In [3]:
input_tensor = ['x_input', 'y_input']
output_tensor = 'z_output'

In [4]:
import collections
node_dict = {}
topo_map = {}
node_order_list = []
topo_map = collections.OrderedDict()
node_dict=collections.OrderedDict() 

In [5]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('./new_ckpt_dir_3/model-10000.meta')
    new_saver.restore(sess, './new_ckpt_dir_3/model-10000')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./new_ckpt_dir_4/model-2000


In [6]:
with tf.Session() as sess:
    for n in sess.graph_def.node:
        node_dict[n.name] = n
        node_order_list.append(n.name)

In [7]:
node_order_list.reverse()

In [16]:
print (node_dict['rnn/strided_slice_1']) # Last Operation

name: "rnn/strided_slice_1"
op: "StridedSlice"
input: "rnn/Shape_1"
input: "rnn/strided_slice_1/stack"
input: "rnn/strided_slice_1/stack_1"
input: "rnn/strided_slice_1/stack_2"
attr {
  key: "Index"
  value {
    type: DT_INT32
  }
}
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}
attr {
  key: "begin_mask"
  value {
    i: 0
  }
}
attr {
  key: "ellipsis_mask"
  value {
    i: 0
  }
}
attr {
  key: "end_mask"
  value {
    i: 0
  }
}
attr {
  key: "new_axis_mask"
  value {
    i: 0
  }
}
attr {
  key: "shrink_axis_mask"
  value {
    i: 1
  }
}



#### 1. Define muti-tree data structure

In [9]:
class Node:
    
    __children = []
    __tensor = None
    __shape = None
    
    def __init__(self, name = None, child_list = [], parent = None, Op = None):
        self.Op = Op
        self.name = name
        self.parent = parent
        for elem in child_list:
            self.__children.append(elem)
    
    def get_name(self):
        return self.name
    
    def get_child(self):
        return self.__children
    
    def get_Op(self):
        return self.Op
    
    def get_parent(self):
        return self.parent
    
    def get_tensor(self):
        return self.__tensor
    
    def get_shape(self):
        return self.__shape
    
    def set_shape(self, shape):
        self.__shape = shape 
    
    def set_tensor(self, tensor):
        self.__tensor = tensor
    
    def set_parent(self, parent):
        self.parent = parent
    
    def set_Op(self, Op):
        self.Op = Op
        
    def set_name(self, name):
        self.name = name
    
    def set_child(self, chil):
        self.__children = []
        if chil == None:
            return
        for elem in chil:
            self.__children.append(elem)

#### 2. Store all necessary node in graph

In [10]:
# Construct child node list
def Construct_Node_List(name_list, parent, node_dict):
    # name_list: node_name of each graph node
    # node_dict: key:node name; value: Node object
    chil = []
    if len(name_list) == 0 or (parent != None and parent.get_name().startswith('Variable') and parent.get_name().endswith('read')):
        return None
    else:
        for elem in name_list:
            print (elem)
            chil.append(Node(elem, [], parent, node_dict[elem].op))
    return chil

In [11]:
root = Node(output_tensor, Construct_Node_List(node_dict[output_tensor].input, None, node_dict), None, node_dict[output_tensor].op)

add


In [12]:
print (root.get_child()[0].get_name())

add


In [17]:
print (node_dict['rnn/while/Enter_2'])

name: "rnn/while/Enter_2"
op: "Enter"
input: "rnn/TensorArray:1"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "frame_name"
  value {
    s: "rnn/while/while_context"
  }
}
attr {
  key: "is_constant"
  value {
    b: false
  }
}
attr {
  key: "parallel_iterations"
  value {
    i: 32
  }
}



#### 2.1 Construct Multi Tree 

In [13]:
import queue
Q_1 = queue.Queue()
Q_1.put(root)
print(Q_1.qsize())
while (Q_1.qsize() != 0):
    temp_node = Q_1.get()
    print ('---------------')
    print (temp_node.get_name())
    # update child
    print ('***************')
    temp_node.set_child(Construct_Node_List(node_dict[temp_node.get_name()].input, temp_node, node_dict))
    print ('***************')
    # update queue
    for elem in temp_node.get_child():
        Q_1.put(elem)
# root.get_child()[0].get_child()[0].get_Op()
def Loop_Multi_Tree(root):
    if (len(root.get_child()) == 0):
        # which means that it is one leaf node
        indegree_node[root] = 0
        return
    indegree_node[root] = len(root.get_child())
    for each_child_node in root.get_child():
        print (each_child_node.get_name())
        Loop_Multi_Tree(each_child_node)
indegree_node = {}
Loop_Multi_Tree(root)

1
---------------
z_output
***************
add
***************
---------------
add
***************
MatMul
biases/b/read
***************
---------------
MatMul
***************
strided_slice
weights/W/read
***************
---------------
biases/b/read
***************
biases/b
***************
---------------
strided_slice
***************
rnn/transpose_1
strided_slice/stack
strided_slice/stack_1
strided_slice/stack_2
***************
---------------
weights/W/read
***************
weights/W
***************
---------------
biases/b
***************
***************
---------------
rnn/transpose_1
***************
rnn/TensorArrayStack/TensorArrayGatherV3
rnn/concat_2
***************
---------------
strided_slice/stack
***************
***************
---------------
strided_slice/stack_1
***************
***************
---------------
strided_slice/stack_2
***************
***************
---------------
weights/W
***************
***************
---------------
rnn/TensorArrayStack/TensorArrayGathe

KeyError: 'rnn/TensorArray:1'

#### 3. Calculate the degree of all nodes

#### 4. Topo Sort

In [14]:
import queue

In [15]:
q_topo = queue.Queue()
for each_node in indegree_node:
    if indegree_node[each_node] == 0:
        q_topo.put(each_node)

In [16]:
print (q_topo.qsize())

12


In [17]:
def Construct_topo_json(q_topo, graph, index, data, sess):
    while (q_topo.qsize() != 0):
        cur_size = q_topo.qsize()
        step = "step" + str(index)
        unit = {}
        while cur_size > 0:
            single_op = {}
            temp_node = q_topo.get()
            tensor = graph.get_tensor_by_name(temp_node.get_name()+":"+'0')
            if temp_node.get_parent() != None:
                indegree_node[temp_node.get_parent()] = indegree_node[temp_node.get_parent()] - 1
                if (indegree_node[temp_node.get_parent()] == 0):
                    q_topo.put(temp_node.get_parent())
            if temp_node.get_name().startswith('Variable') and temp_node.get_Op() == 'Identity':
                print ('We should get the actual tensor value of the node *** %s ***' % temp_node.get_name())
                single_op["TYPE"] = temp_node.get_name()
                tensor_value = tensor.eval(session=sess)
                if type(tensor_value) != np.ndarray:
                    tensor_value = np.array(tensor_value)
                single_op["TENSOR_VALUE"] = tensor_value.tolist()
                single_op["OPERATION"] = temp_node.get_Op()
                single_op["SHAPE"] = str(tensor.shape)
                single_op["STRIDE"] = None
                single_op["PADIING"] = None
                single_op["INPUT"] = []
                single_op["FATHER"] = temp_node.get_parent().get_name() if temp_node.get_parent()!= None else None
            elif temp_node.get_Op() == 'Placeholder':
                print ('We should fullfill the tensor *** %s *** with input value' % temp_node.get_name())
                single_op["TYPE"] = temp_node.get_name()
                single_op["TENSOR_VALUE"] = None
                single_op["OPERATION"] = temp_node.get_Op()
                single_op["SHAPE"] = str(tensor.shape)
                single_op["STRIDE"] = None
                single_op["PADIING"] = None
                single_op["INPUT"] = []
                single_op["FATHER"] = temp_node.get_parent().get_name() if temp_node.get_parent()!= None else None
            elif len(temp_node.get_child()) > 0:
                # We should note the actual operations
                print ('Do %s operation' % temp_node.get_Op())
                single_op["TYPE"] = temp_node.get_name()
                single_op["TENSOR_VALUE"] = None
                single_op["OPERATION"] = temp_node.get_Op()
                single_op["SHAPE"] = str(tensor.shape) if temp_node.get_Op() == 'Reshape' or temp_node.get_Op() == 'Conv2D' else None
                if (temp_node.get_Op() == 'Conv2D'):
                    single_op["STRIDE"] = str(tuple([int(elem) for elem in node_dict[temp_node.get_name()].attr['strides'].list.i]))
                    single_op["PADIING"] = bytes.decode(node_dict[temp_node.get_name()].attr['padding'].s)
                else:
                    single_op["STRIDE"] = None
                    single_op["PADIING"] = None
                input_list = []
                for each_input_node in temp_node.get_child():
                    print("My name is *** %s *** and I am the input of *** %s ***" % (each_input_node.get_name(), each_input_node.get_parent().get_name()))
                    input_list.append(each_input_node.get_name())
                single_op["INPUT"] = input_list
                single_op["FATHER"] = temp_node.get_parent().get_name() if temp_node.get_parent()!= None else None
            else:
                print ('Do %s operation' % temp_node.get_Op())
                single_op["TYPE"] = temp_node.get_name()
                single_op["TENSOR_VALUE"] = None
                single_op["OPERATION"] = temp_node.get_Op()
                single_op["SHAPE"] = str(tensor.shape)
                single_op["STRIDE"] = None
                single_op["PADIING"] = None
                single_op["INPUT"] = []
                single_op["FATHER"] = temp_node.get_parent().get_name() if temp_node.get_parent()!= None else None
            unit[temp_node.get_name()] = single_op
            cur_size = cur_size - 1
        index = index + 1
        data[step] = unit
    return data

In [18]:
import json

In [19]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('./new_ckpt_dir_3/model-10000.meta')
    new_saver.restore(sess, './new_ckpt_dir_3/model-10000')
    graph = tf.get_default_graph()
    from collections import OrderedDict
    data = OrderedDict()
    index = 0
    data = Construct_topo_json(q_topo, graph, index, data, sess)
    with open ("model6.json", "w") as fp:
        fp.write(json.dumps(data, indent=4))

INFO:tensorflow:Restoring parameters from ./new_ckpt_dir_3/model-10000
We should fullfill the tensor *** x_input *** with input value
We should get the actual tensor value of the node *** Variable_1/read ***
We should get the actual tensor value of the node *** Variable_2/read ***
We should get the actual tensor value of the node *** Variable_3/read ***
We should get the actual tensor value of the node *** Variable_4/read ***
We should get the actual tensor value of the node *** Variable_5/read ***
We should get the actual tensor value of the node *** Variable_6/read ***
Do Const operation
We should get the actual tensor value of the node *** Variable_7/read ***
We should get the actual tensor value of the node *** Variable_8/read ***
We should get the actual tensor value of the node *** Variable_9/read ***
We should get the actual tensor value of the node *** Variable_10/read ***
Do Conv2D operation
My name is *** x_input *** and I am the input of *** Conv2D ***
My name is *** Variabl